In [8]:
# Import all Libraries and set path varaibles 
import numpy as np
import pandas as pd
from restaurant_index import restaurant_index
from crime_index import crime_index
from transit_index import transit_index
from housing_index import housing_index
from health_index import health_index
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


#USER SET VARIABLES 
path="team104final/Data/"
test_files="Test_files/"
RAW_RUN=False


RAW RUN 

In [9]:
## This section will run with RAW data and calulate each individual index. 

if RAW_RUN==True:
    restaurant_index_df=restaurant_index(path+"DOHMH_New_York_City_Restaurant_Inspection_Results_20231025.csv")
    restaurant_index_df.to_csv(test_files+'restaurant_index_df.csv',index=False)

    crime_index_df=crime_index(path+"Crime_Map_.csv")
    crime_index_df.to_csv(test_files+'crime_index_df.csv',index=False)

    transit_index_df=transit_index(subway_data=path+'MTA_NYCT_Subway_Entrances_and_Exits__2015_20231113.csv', bus_data=path+'Bus_Stop_Shelter.csv', bike_data=path+'BicycleParking.csv')
    transit_index_df.to_csv(test_files+'transit_index_df.csv')

    health_index_df=health_index(water_inspection_data=path+"Water_inspection_data.csv",Rodent_Inspection_data=path+"Rodent_Inspection_data.csv",Influenza_Pneumonia_data=path+"Influenza_Pneumonia_data.csv",EMS_Incident_data=path+"EMS_Incident_data.csv")
    health_index_df.to_csv(test_files+"health_index_df.csv",index=False)


    housing_index_df=housing_index(projects_data=path+'projects_data.csv', buildings_data=path+"buildings_data.csv", rent_data=path+'rent_data.csv', violations_data=path+"violations_data.csv")
    housing_index_df.to_csv(test_files+"housing_index_df.csv",index=False)




    ## TRANSIT WITH CENSUS_GEOID

    # transit_index_df=transit_index(subway_data="Test_files/subway_entrances.csv", bus_data="Test_files/bus_stops.csv", bike_data="Test_files/bike_parking.csv")
    ############




Pre-Processed Run 

In [11]:
##This will run using the pre-processed data saved inside "test_files"

restaurant_index_df=pd.read_csv(test_files+'restaurant_index_df.csv')
restaurant_index_df['restaurant_index']=restaurant_index_df['linear_equation']
restaurant_index_df=restaurant_index_df[['census_tract_geoid','restaurant_index']]


crime_index_df=pd.read_csv(test_files+"crime_index_df.csv")
crime_index_df['crime_index']=crime_index_df['StandardizedScore']
crime_index_df=crime_index_df[['census_tract_geoid','crime_index']]

transit_index_df=pd.read_csv(test_files+"transit_index_df.csv")
transit_index_df['transit_index']=transit_index_df['index_score']
transit_index_df=transit_index_df[['census_tract_geoid','transit_index']]

health_index_df=pd.read_csv(test_files+"health_index_df.csv")
health_index_df[['health_index']]=health_index_df[['Health_Index']]
health_index_df=health_index_df[['census_tract_geoid','health_index']]

housing_index_df=pd.read_csv(test_files+"housing_index_df.csv")
housing_index_df[['housing_index']]=housing_index_df[['index_score']]
housing_index_df=housing_index_df[['census_tract_geoid','housing_index']]


Joins

In [12]:
# This joins all indexes together
join1=restaurant_index_df.merge(crime_index_df,on="census_tract_geoid",how='outer')
join2=join1.merge(transit_index_df,on='census_tract_geoid',how='outer')
join3=join2.merge(health_index_df,on='census_tract_geoid',how='outer')
join4=join3.merge(housing_index_df,on='census_tract_geoid',how='outer')

join4.isnull().sum(axis = 0)

census_tract_geoid       0
restaurant_index       360
crime_index              2
transit_index          302
health_index          2031
housing_index           73
dtype: int64

In [6]:
#This removes rows with >3 missing indexes 
row_keep=[]
for i in range(len(join4)):
    if join4.iloc[i].isna().sum()<3:
        row_keep.append(i)
rows_underThree_na=join4.iloc[row_keep]
rows_underThree_na.isnull().sum(axis = 0)


census_tract_geoid       0
restaurant_index       190
crime_index              0
transit_index          142
health_index          1852
housing_index           20
dtype: int64

In [7]:
#This imputes missing data for missing indexes and outputs to csv
imp = IterativeImputer(max_iter=10, random_state=0)
final_noNa=rows_underThree_na.copy()
missing_data=['restaurant_index','health_index','transit_index','crime_index','housing_index']
final_noNa[missing_data] = imp.fit_transform(final_noNa[missing_data])
final_index=final_noNa.copy()
final_index.to_csv(test_files+'Final/final_index.csv',index=False)



